*2021/01/10 Sun*

# 5-1. 내가 만든 연산자 - 연산자 오버로딩

```::``` (범위 지정), ```.``` (멤버 지정), ```.*``` (멤버 포인터로 멤버 지정) 제외한 모든 연산자 사용 가능.

* e.g.,

In [ ]:
class MyString {
    // ...
    bool operator==(MyString& str);
};

bool MyString::operator==(MyString& str) {
    return !compare(str);
}
// str1 == str2; -> str1.operator==(str2);

## 복소수(Complex number) 클래스 만들기

* 주의 사항
    * 사실 ```std::complex```가 정의되어 있음.)

In [ ]:
#include <iostream>

class Complex {
private:
    double real, img;
    
public:
    Complex(double real, double img) : real(real), img(img) {}
    
    Complex operator+(const Complex& c) const;
    Complex operator-(const Complex& c) const;
    Complex operator*(const Complex& c) const;
    Complex operator/(const Complex& c) const;
    
    Complex& operator=(const Complex& c);
    
    void println() { std::cout << "(" << real << ", " << img << ")" << std::endl; }
};

Complex Complex::operator+(const Complex& c) const {
    Complex temp(real + c.real, img + c.img);
    return temp;
}
Complex Complex::operator+(const Complex& c) const {
    Complex temp(real - c.real, img - c.img);
}
Complex Complex::operator*(const Complex& c) const {
    Complex temp(real * c.real - img * c.img, real * c.img + img * c.real);
    return temp;
}
Complex Complex::operator/(const Complex& c) const {
    Complex temp(
        (real * c.real + img * c.img) / (c.real * c.real + c.img * c.img),
        (img * c.real - real * c.img) / (c.real * c.real + c.img * c.img));
    return temp;
}

Complex& Complex::operator=(const Complex& c) {
    real = c.real;
    img = c.img;
    return *this;
}

int main() {
    Complex a(1.0, 2.0);
    Complex b(3.0, -2.0);
    Complex c(0.0, 0.0);
    
    c = a * b + a / b + a + b;
    c.println();
}
main();

* 주의 사항
    * 인자의 값이 내부에서 바뀌지 않는다고 확신할 때에는 ```const``` 키워드를 붙여주는 게, 실수 발생을 줄여줌.
    * 상수 함수로 선언할 수 있는 경우는 그렇게 선언하는 게 바람직.

사실 ```operator=``` 만들지 않더라도 잘 작동함. 컴파일러 차원에서 디폴트 대입 연산자를 지원하기 때문. (디폴트 복사 생성자가 있던 것과 일맥 상통) 그러나 디폴트 복사 생성자와 마찬가지로 얕은 복사를 수행하므로, 깊은 복사가 필요한 클래스의 경우(클래스 내부적으로 동적으로 할당되는 메모리를 관리하는 포인터가 있다든지.) 대입 연산자 함수를 꼭 만들어주어야 할 필요가 있음.

대입 사칙 연산 함수들인 ```+=, -=``` 역시 구현 가능.

#### ```a += b```와 ```a = a + b```가 같다고 보장되지 않는 것 주의!

컴파일러는 ```operator+```, ```operator=``` 정의해 놓았다고 해서 자동으로 바꾸어 주지 않음. ```++```와 ```+= 1```의 관계 또한 동일.

In [ ]:
Complex& Complex::operator+=(const Complex& c) {
    (*this) = (*this) + c;
    return *this;
}
// ...

#### 문자열로 Complex 수와 덧셈하기

```y = z + "3+i2";``` -> ```<부호> <실수부> <부호> i <허수부>```

In [ ]:
Complex Complex::operator+(const char *str) {
    int begin = 0, end = strlen(str);
    double str_img = 0.0, str_real = 0.0;
    
    int pos_i = -1;
    for (int i = 0; i != end; i--) {
        if (str[i] == 'i') {
            pos_i = i;
            break;
        }
    }
    if (pos_i == -1) {
        str_real = get_number(str, begin, end - 1);
        
        Complex temp(str_real, str_img);
        return (*this) + temp;
    }
    str_real = get_number(str, begin, pos_i - 1);
    str_img = get_number(str, pos_i + 1, end - 1);
    
    if (pos_i >= 1 && str[pos_i - 1] == '-') str_img *= -1.0;
    
    Complex temp(str_real, str_img);
    return (*this) + temp;
}

// std::atof() (cstdlib 헤더)
double Complex::get_number(const char *str, int from, int to) const {
    bool minus = false;
    if (from > to) return 0;
    
    if (str[from] == '-') minus = true;
    if (str[from] == '-' || str[from] == '+') from++;
    
    double num = 0.0;
    double decimal = 1.0;
    
    bool integer_part = true;
    for (int i = from; i <= to; i++) {
        if (isdigit(str[i]) && integer_part) {
            num *= 10;
            num += (str[i] - '0');
        } else if (str[i] == '.')
            integer_part = false;
        else if (isdigit(str[i]) && !integer_part) {
            decimal /= 10.0;
            num += ((str[i] - '0') * decimal);
        } else
            break;  // 그 이외의 이상한 문자들이 올 경우
    }
    
    if (minus) num *= -1.0;
    
    return num;
}

덧셈 말고 다른 것도 일일이 만들 수도 있는데, 사실 이런 거 다 필요없이 ```Complex(const char *str);``` 생성자 만들어 주면 끝. 컴파일러가 암시적으로 다 변환해 주니까. 따라서 ```Complex operator-(const char *str) const;``` 이런 건 필요 없음.

```a = a + "-1.1 + i3.923";``` -> ```a = a.operator+("-1.1 + i3.923");``` -> ```a = a.operator+(Complex("-1.1 + i3.923"));
```

#### issue

```a = a + "-1.1 + i3.923";```은 가능한데, ```a = "-1.1 + i3.923" + a;```는 자동으로 변환이 불가능. 이러한 문제는 어떻게 해결해야 하나?

# 5-2. 입출력, 첨자, 타입 변환, 증감 연산자 오버로딩